In [1]:

import model_utils
import torch
from models.VGG16 import vgg16
from models.ResNet import ResNet, ResBlock
from models.googlenet import GoogLeNet

加载预训练 VGG16 模型并测试分割功能

In [2]:
# 加载 VGG16 模型
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=False)
# model = vgg16()
model = GoogLeNet()
input_shape = (1, 3, 224, 224)
block_configs, pooling_layers = model_utils.auto_segment_model(model, input_shape)
print('分割得到的块配置:')
for k, v in block_configs.items():
    print(f'{k}: {v}')
print('识别出的池化层:')
for i, pool in enumerate(pooling_layers):
    print(f'Pool {i+1}: {type(pool)}')

警告: 模型没有 'features' 属性或该属性不是 nn.Sequential。无法自动分割。
分割得到的块配置:
识别出的池化层:


In [3]:
import torch
import sys

# 确保 model_utils 和 model_segments 可以在 Python 路径中被找到
try:
    from model_utils import auto_segment_model
    from model_segments import load_segment_models_dynamically
except ImportError:
    print("错误: 请确保 'model_utils.py' 和 'model_segments.py' 文件与此测试脚本在同一目录下。")
    sys.exit(1)


In [4]:

def run_test_for_model(model_name, input_shape, k, r):
    """
    对指定的模型运行一套完整的分割和加载测试。
    """
    print(f"\n{'='*20} 开始测试: {model_name.upper()} {'='*20}")
    
    try:
        print(f"--> 正在调用 load_segment_models_dynamically for '{model_name}'...")
        master_models, worker_models, pooling_layers = load_segment_models_dynamically(
            model_name=model_name,
            k_workers=k,
            r_workers=r,
            input_shape=input_shape
        )
        print("--> 动态加载成功！")

        num_blocks = len(master_models)
        print(f"\n--- 验证结果 ---")
        print(f"识别出的卷积块数量: {num_blocks}")
        print(f"识别出的池化层数量: {len(pooling_layers)}")
        
        assert num_blocks > 0, f"测试失败: 未能为 '{model_name}' 识别出任何卷积块。"
        assert len(master_models) == len(worker_models), "测试失败: Master 和 Worker 的模型数量不匹配。"
        assert abs(len(master_models) - len(pooling_layers)) <= 1, "测试失败: 池化层和卷积块的数量关系不正确。"

        print("\n--- 详细配置检查 ---")
        for i, block_name in enumerate(master_models.keys()):
            print(f"  - 块 {i+1}: '{block_name}'")
            assert block_name in worker_models, f"测试失败: Worker 模型中缺少 '{block_name}'。"
            
            encoder, final_decoder = master_models[block_name]
            worker_decoder = worker_models[block_name]
            
            assert isinstance(encoder, torch.nn.Module), f"测试失败: '{block_name}' 的 Encoder 不是一个 nn.Module。"
            assert isinstance(final_decoder, torch.nn.Module), f"测试失败: '{block_name}' 的 FinalDecoder 不是一个 nn.Module。"
            assert isinstance(worker_decoder, torch.nn.Module), f"测试失败: '{block_name}' 的 WorkerDecoder 不是一个 nn.Module。"
            print(f"    - Encoder, FinalDecoder, WorkerDecoder 实例已成功创建。")

        print(f"\n{'='*20} 测试成功: {model_name.upper()} {'='*20}")
        return True

    except Exception as e:
        print(f"\n{'!'*20} 测试失败: {model_name.upper()} {'!'*20}")
        print(f"错误信息: {e}")
        import traceback
        traceback.print_exc()
        return False


In [5]:

if __name__ == '__main__':
    k, r = 4, 2
    
    # --- 测试 VGG16 ---
    vgg_success = run_test_for_model('vgg16', (1, 3, 224, 224), k, r)
    
    print("\n" + "-"*50 + "\n")
    
    # --- 测试 AlexNet ---
    alexnet_success = run_test_for_model('alexnet', (1, 3, 224, 224), k, r)

    print("\n" + "#"*50)
    if vgg_success and alexnet_success:
        print("所有测试均已通过！您的 model_segments.py 文件工作正常。")
    else:
        print("部分或全部测试失败。请检查上面的错误日志。")
    print("#"*50)




==================== 开始测试: VGG16 ====================
--> 正在调用 load_segment_models_dynamically for 'vgg16'...
DEBUG: Block 1 Input Shape: [1, 3, 224, 224]
0.004001140594482422
0.007002115249633789
DEBUG: block_1 Conv Output Shape: [1, 64, 224, 224]
DEBUG: block_1 Pool Output Shape: [1, 64, 112, 112]
DEBUG: Block 2 Input Shape: [1, 64, 112, 112]
0.004999876022338867
0.005998373031616211
DEBUG: block_2 Conv Output Shape: [1, 128, 112, 112]
DEBUG: block_2 Pool Output Shape: [1, 128, 56, 56]
DEBUG: Block 3 Input Shape: [1, 128, 56, 56]
0.005000591278076172
0.005999326705932617
0.008004188537597656
DEBUG: block_3 Conv Output Shape: [1, 256, 56, 56]
DEBUG: block_3 Pool Output Shape: [1, 256, 28, 28]
DEBUG: Block 4 Input Shape: [1, 256, 28, 28]
0.00599980354309082
0.008201122283935547
0.004001140594482422
DEBUG: block_4 Conv Output Shape: [1, 512, 28, 28]
DEBUG: block_4 Pool Output Shape: [1, 512, 14, 14]
DEBUG: Block 5 Input Shape: [1, 512, 14, 14]
0.0020003318786621094
0.002000093460083008

Traceback (most recent call last):
  File "C:\Users\Zaoer\AppData\Local\Temp\ipykernel_58632\3380877305.py", line 9, in run_test_for_model
    master_models, worker_models, pooling_layers = load_segment_models_dynamically(
  File "d:\OneDrive\Study\CodedInference\model_segments.py", line 168, in load_segment_models_dynamically
    raise ValueError(f"模型 '{model_name}' 尚不支持自动分割。")
ValueError: 模型 'alexnet' 尚不支持自动分割。
